# Conditional Adaptive Histogram Equalization Example

Comparing the Python and Julia implementations of the CLAHE code.

In [ ]:
using IceFloeTracker, Images, DataFrames
import IceFloeTracker.Filtering: adapthisteq_py, adapthisteq, get_rgb_channels

Use the standard dataset:

In [ ]:
dataset = Watkins2026Dataset()

An example case: the red channel from case 128 (Hudson Bay) the validation dataset. 

We call `adapthisteq` implemented in Julia and `adapthisteq_py` implemented in Python, and compare the results.

In [ ]:
case128red = get_rgb_channels(modis_truecolor(dataset[256]))[:,:,1]
img_eq_jl = adapthisteq(case128red)
img_eq_py = adapthisteq_py(case128red)
difference = img_eq_jl .- img_eq_py
display("case 1")
display(
    hcat(Gray.(case128red / 255.), 
    Gray.(img_eq_py / 255.), 
    Gray.(img_eq_jl / 255.), 
    Gray.(difference / 255 * 5))
    )

In general, we want to compare 
- the status quo Python result to the original, 
- the original to the new implementation, 
- and the difference between the two results.

We use the `compare` function to calculate the following images shown from left to right in the example below:
- Python output
- difference Python vs original,
- original,
- difference Julia vs original,
- Julia output
- difference Julia vs Python.

Differences are shown as offsets from a half-gray tone, so flat gray means no difference. 

In [ ]:
function compare(img; jl_args=(), jl_kwargs=Dict(), py_args=(), py_kwargs=Dict(), difference_offset=0.5, difference_factor=1.0)
    img_eq_jl = adapthisteq(img, jl_args...; jl_kwargs...)
    img_eq_py = adapthisteq_py(img, py_args...; py_kwargs...)
    
    result = (
        py=Gray.(img_eq_py / 255.), 
        im_diff_py=Gray.(0.5 .+(( img .- img_eq_py) / 255 * difference_factor)),
        im=Gray.(img / 255.), 
        im_diff_jl = Gray.(0.5 .+(( img .- img_eq_jl) / 255 * difference_factor)),
        jl=Gray.(img_eq_jl / 255.), 
        jl_diff_py=Gray.(0.5 .+(( img_eq_jl .- img_eq_py) / 255 * difference_factor)),
    )
    return result
    
end

display(hcat(compare(case128red, jl_kwargs=Dict(:nbins=>256, :clip=>0.08, :rblocks=>8, :cblocks=>8))...))

We next look at the full validation dataset. 
We want to find cases where the Julia output is maximally different from the Python output. 
To find cases which are perceptually different, we use the structural similarity index measure (SSIM),
where 1 is identical and 0 is completely different.

In [ ]:
results = []
for case in dataset
    img = modis_truecolor(case)
    rgbchannels = get_rgb_channels(img)
    channel1 = rgbchannels[:, :, 1]
    img_results = compare(channel1; jl_kwargs=Dict(:nbins => 256, :clip => 0.1))

    img_eq_jl = img_results.jl
    img_eq_py = img_results.py

    result = (name=name(case),
        ssim_original_py=assess_ssim(channel1, img_eq_py),
        ssim_original_jl=assess_ssim(channel1, img_eq_jl),
        ssim_jl_py=assess_ssim(img_eq_jl, img_eq_py), 
        img_results...)
    push!(results, result)
end
results_df = DataFrame(results);

The "worst" cases are shown below.

The Julia implementation makes more blocking artifacts, 
particularly in low-contrast areas of open water, 
or on artifact edges.
Performance in the sea-ice regions looks similar. 

Tuning the parameters of the Julia implementation 
to match the Python implementation more closely
was found to be difficult. 
I tried tuning the `clip` and `nbins` parameters.
Changing the default `clip` parameter to 0.1 rather than 0.01 seemed better in some cases. 

The image order is:
- Original
- Python result,
- Julia result,
- Python vs Julia difference, scaled as differences around half-gray.

In [ ]:
for row in eachrow(sort(results_df, :ssim_jl_py))[1:20]
    display(row.name)
    display(hcat(row.im, row.py, row.jl, row.jl_diff_py))
end